In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

fifa_df = pd.read_csv('../preprocessing/clean_data_normalized.csv')
fifa_df.head()

In [ ]:
gk_columns = [c for c in fifa_df.columns if 'gk' in c]
fifa_df.drop(gk_columns, axis='columns', inplace=True)

original_positions = 'rs, rw, rf, ram, rcm, rm, rdm, rcb, rb, rwb, st, lw, cf, cam, cm, lm, cdm, cb, lb, lwb, ls, lf, lam, lcm, ldm, lcb, gk'
original_positions = original_positions.replace(' ', '').split(',')

#Filtrando posições que não foram eliminadas pelo filtro de correlação
positions = [p for p in original_positions if p in fifa_df.columns]
positions

In [ ]:
# Define a posição do jogador com base na que ele é melhor e cria uma coluna nova p/ armazenar isso
fifa_df['best_pos'] = fifa_df[positions].idxmax(axis='columns')
#positions_df = fifa_df[positions]
# Apaga as outras informações sobre a performance do jogador em cada posição
fifa_df.drop(positions, axis='columns', inplace=True)

In [ ]:
# Escolha do modelo de clustering
from sklearn.cluster import KMeans
#from  sklearn.cluster import AgglomerativeClustering
#from  sklearn.cluster import DBSCAN
#from sklearn.cluster import SpectralClustering
from sklearn.model_selection import cross_val_score

kmeans = KMeans(n_clusters=len(positions), max_iter=1000)

# Como vai agrupar p/ descobrir a posição, tira a posição do dataset de treinamento
data_X = fifa_df.drop('best_pos', axis='columns')
# Agrupa os dados e pega o grupo de cada amostra
kmeans_labels = kmeans.fit_predict(data_X)

# Cria uma nova coluna com os grupos preditos pelo kmeans
fifa_df['kmeans_labels'] = pd.Series(kmeans_labels)

In [ ]:
# Agrupa o dataset pelas labels obtidas por clustering e pega a quantidade jogadores em cada posição
clusters_positions = fifa_df.groupby('kmeans_labels')['best_pos'].value_counts().to_dict()
clusters_positions

In [ ]:
# A label do cluster é a posição mais frequente nele. As 3 células abaixo calculam a posição dos jogadores de cada grupo
position2cluster = {}
curr_max = {k:0 for k in positions}
cp = {}

for k, v in clusters_positions.items():
    cluster, pos = k
    candidates = position2cluster.get(pos, {})
    candidates[cluster] = v
    position2cluster[pos] = candidates
    
len(positions), len(position2cluster.keys())

In [ ]:
df = pd.DataFrame(position2cluster)
max_values = df.idxmax().to_dict()
max_values

In [ ]:
final_clusters = {p:0 for p in df.columns}

for pos in final_clusters:
    final_clusters[pos] = max_values[pos]
    df.drop(max_values[pos], axis=0, inplace=True)
    df.drop(pos, axis='columns', inplace=True)
    if len(df)>0:
        max_values = df.idxmax().to_dict()
    print(len(df))

final_clusters

In [ ]:
cluster2name = inv_map = {v: k for k, v in final_clusters.items()} # invertendo o dict
fifa_df.kmeans_labels.replace(cluster2name, inplace=True)
(fifa_df.kmeans_labels == fifa_df.best_pos).describe()

In [ ]:
# Reduzir as features em 2 para ver a distribuição de grupos
from sklearn.decomposition import PCA
pca = PCA(2)
data_pca = fifa_df.drop(['best_pos', 'kmeans_labels'], axis='columns')
data_pca = pca.fit_transform(data_pca)

data_pca = pd.DataFrame(data_pca, columns=['x','y'])
data_pca['label'] = fifa_df.kmeans_labels
data_pca.head()

In [ ]:
import seaborn as sns
#plot data with seaborn
group_plot = sns.lmplot(data=data_pca, x='x', y='y', hue='label', fit_reg=False, legend=False, legend_out=True)
plt.title("Agrupamento dos Jogadores por Posição")
plt.legend().set_title("Posição")

In [ ]:
# Reduzir as features em 3 para ver a distribuição de grupos
from sklearn.decomposition import PCA
pca = PCA(3)
data_pca = fifa_df.drop(['best_pos', 'kmeans_labels'], axis='columns')
data_pca = pca.fit_transform(data_pca)

data_pca = pd.DataFrame(data_pca, columns=['x','y', 'z'])
data_pca['label'] = fifa_df.kmeans_labels
data_pca.head()

In [ ]:
from mpl_toolkits import mplot3d
colors = {
    'rw': 'red',
    'rdm': 'green',
    'rs': 'blue',
    'rcm': 'gray',
    'rcb': 'purple'
}
data_pca['color'] = pd.Series(colors[label] for label in data_pca.label)
fig = plt.figure()
ax = plt.axes(projection='3d')

ax.scatter(data_pca['x'], data_pca['y'], data_pca['z'], c=data_pca['color'], alpha=1, linewidth=0.5, s=30, edgecolors='none')

df = data_pca.drop_duplicates('label')
for index, row in df.iterrows():
    ax.scatter(row['x'], row['y'], row['z'], c=row['color'] , label=row['label'], alpha=1,linewidth=0.5, s=30, edgecolors='none')

#plt.legend(bbox_to_anchor=(1, 1), loc=2, borderaxespad=0.)
plt.title('Agrupamento dos Jogadores por Posição')
plt.legend(bbox_to_anchor=(1., .7), loc=2, borderaxespad=0.).set_title('Posição')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.view_init(45, 200)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(fifa_df.kmeans_labels, fifa_df.best_pos)
cm = pd.DataFrame(cm, columns=positions, index=positions)

heatmap = sns.heatmap(cm, annot=True, fmt='d',cbar_kws={'label': 'Quantidade de Jogadores'})
heatmap.yaxis.set_ticklabels(positions, rotation=0, ha='right', fontsize=10)
heatmap.xaxis.set_ticklabels(positions, rotation=0, ha='right', fontsize=10)

plt.ylabel('Posição Predita')
plt.xlabel('Posição Real')
plt.title("Matriz de Confusão - Posição Predita vs Posição Real")